In [1]:
# randomly xxxx
import numpy as np 
np.random.seed(42)


In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing(as_frame=True)
data = housing['data']
target = housing['target']


In [3]:
#spliting data
import numpy as np 
import pandas as pd 

def split_train_test(data , test_ratio ):
    shuffled= np.random.permutation(len(data))
    test_set_size = int(len(data)* test_ratio)
    test= shuffled[:test_set_size]
    train = shuffled[test_set_size:]
    return data.iloc[train], data.iloc[test]

housing = pd.concat([data, target] , axis=1)


train , test = split_train_test(housing , 0.2)    

In [4]:
# orrrrrr working with hash
#better solution 
from zlib import crc32

def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

housing_with_id = housing.reset_index() # index
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")


In [5]:
#$simpler
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)


In [6]:
print(housing.columns)


Index(['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup',
       'Latitude', 'Longitude', 'MedHouseVal'],
      dtype='object')


In [7]:
#Stratified Sampling

housing["income_cat"] = np.ceil(housing["MedInc"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)



/tmp/ipykernel_1193/2814207913.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)


In [8]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]


In [9]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)
